In [2]:
!pip install -r requirements.txt

import os
if os.path.exists("new_data.zip"):
    !unzip -o new_data.zip

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Archive:  new_data.zip
  inflating: __MACOSX/._new_data     
  inflating: new_data/B0007GFDP4.json  
  inflating: __MACOSX/new_data/._B0007GFDP4.json  
  inflating: new_data/B000162MSG.json  
  inflating: __MACOSX/new_data/._B000162MSG.json  
  inflating: new_data/B000FDBQHS.json  
  inflating: __MACOSX/new_data/._B000FDBQHS.json  
  inflating: new_data/B000EMEEHE.json  
  inflating: __MACOSX/new_data/._B000EMEEHE.json  
  inflating: new_data/B000E671W0.json  
  inflating: __MACOSX/new_data/._B000E671W0.json  
  inflating: new_data/B000084E76.json  
  inflating: __MACOSX/new_data/._B000084E76.json  
  inflating: new_data/B0009TQC90.json  
  inflating: __MACOSX/new_data/._B0009TQC90.json  
  inflating: new_data/B0001FR0B6.json  
  inflating: __MACOSX/new_data/._B0001FR0B6.json  
  inflating: new_data/B000FPGYTQ.json  
  inflating: __MACOSX/new_data/._B000FPGYTQ.json  
  inflating: new_data/B00061ETWS.json  
  inflating: __MACO

In [3]:
from amazon_new_data_scraping import parallel_scrape_amazon_product_info
import pandas as pd

df_amazon_food_reviews = pd.read_csv('amazon_food_reviews.csv')

In [4]:
product_id_list = list(df_amazon_food_reviews.ProductId.unique())
print(f"Total number of products: {len(product_id_list)}")
print("Start scraping data...")
parallel_scrape_amazon_product_info(product_id_list, replace=False)
print("Scraping finished.")

Total number of products: 4322
Start scraping data...
Using 12 CPU cores for parallel scraping


Scraping amazon products: 100%|██████████| 4322/4322 [00:01<00:00, 3872.82it/s]

Successfully scraped 4322/4322 products
Scraping finished.
